In [1]:
import sys
sys.path.append('..')

import os
import pandas as pd 
import numpy as np

from config import *
from helpers.analysis import *

TABLES_DIR = os.path.join(paper, 'tables')
FIG_DIR = os.path.join(paper, 'figs')
for dir in [TABLES_DIR, FIG_DIR]:
    if not os.path.exists(dir):
        os.makedirs(dir)

maestro_subset_meta = pd.read_csv(EVAL_DATA_PATHS['maestro_subset']['meta'])

In [2]:
# Table 1: Evaluation set statistics
meta = pd.concat([maestro_subset_meta], ignore_index=True)

composers = meta.groupby(['composer'])['title'].nunique().index
num_pieces = meta.groupby(['composer'])['title'].nunique().values.astype(int)
num_performances = meta.groupby(
    ['composer'])['title'].count().values.astype(int)
duration_min = np.round(
    ((meta.groupby(['composer'])['duration_sec'].sum() / 60).values), 2)

dataset_table_columns = ['composer', 'pieces',
                         'performances', 'duration (min)']
dataset_table_arrays = [composers, num_pieces, num_performances, duration_min]
eval_set = pd.DataFrame({k: v for k, v in zip(dataset_table_columns, dataset_table_arrays)
                         })
# add total row
eval_set.loc['Total'] = eval_set.sum(numeric_only=True)
eval_set.fillna('Total', inplace=True)
eval_set
# prettify
eval_set['pieces'] = eval_set['pieces'].astype(int)
eval_set['performances'] = eval_set['performances'].astype(int)
# # save to latex
eval_set.to_latex(f'{TABLES_DIR}/tab1_eval_set.tex', index=False,
                  escape=False, float_format="%.2f")

eval_set

,composer,pieces,performances,duration (min)
0,Bach,1,7,23.36
1,Beethoven,5,28,285.54
2,Chopin,4,15,150.28
3,Debussy,2,3,32.06
4,Glinka,1,2,10.35
5,Haydn,3,9,90.23
6,Liszt,3,12,58.98
7,Mozart,2,4,29.02
8,Rachmaninoff,2,3,11.87
9,Schubert,3,17,107.27


In [3]:
# Table 2 : Metrics Overview per model and metric for MAESTRO subset
frame_res = pd.read_csv(os.path.join(results, 'IR_frame.csv'))
note_offset_res = pd.read_csv(os.path.join(results, 'IR_note_offset.csv'))
note_offset_velocity_res = pd.read_csv(
    os.path.join(results, 'IR_note_offset_velocity.csv'))

timing_res = pd.read_csv(os.path.join(results, 'musical_timing.csv'))
articulation_res = pd.read_csv(os.path.join(results, 'musical_articulation.csv'))
harmony_res = pd.read_csv(os.path.join(results, 'musical_harmony.csv'))
dynamics_res = pd.read_csv(os.path.join(results, 'musical_dynamics.csv'))

frame_metrics = ['f_f']
note_off_metrics = ['f_no']
note_off_vel_metrics = ['f_nov']
timing_metrics = ['melody_ioi_corr', 'acc_ioi_corr']
articulation_metrics = ['melody_kor_corr_64', 'bass_kor_corr_64', 'ratio_kor_corr_64']
harmony_metrics = ['cd_corr', 'cm_corr', 'ts_corr']
dynamics_metrics = ['dyn_corr']

frame_res.shape, note_offset_res.shape, note_offset_velocity_res.shape, timing_res.shape, articulation_res.shape, harmony_res.shape, dynamics_res.shape

peamt = pd.read_csv(os.path.join(results, 'peamt.csv'))

# combine 
ALL_RES_MAESTRO = frame_res.copy()
ALL_RES_MAESTRO.drop(columns=['title', 'performer', 'p_f', 'r_f'], inplace=True)
ALL_RES_MAESTRO[note_off_metrics] = note_offset_res[note_off_metrics]
ALL_RES_MAESTRO[note_off_vel_metrics] = note_offset_velocity_res[note_off_vel_metrics]
ALL_RES_MAESTRO[timing_metrics] = timing_res[timing_metrics]
ALL_RES_MAESTRO[articulation_metrics] = articulation_res[articulation_metrics]
ALL_RES_MAESTRO[harmony_metrics] = harmony_res[harmony_metrics]
ALL_RES_MAESTRO[dynamics_metrics] = dynamics_res[dynamics_metrics]
ALL_RES_MAESTRO['peamt'] = peamt['peamt']
# filter to get only MAESTRO
ALL_RES_MAESTRO = ALL_RES_MAESTRO[ALL_RES_MAESTRO['recording'] == 'maestro']
ALL_RES_MAESTRO.drop(columns=['recording'], inplace=True)     
ALL_RES_MAESTRO.columns

all_metrics = ALL_RES_MAESTRO.columns.values[3:]
# sort by split
MAESTRO_RES_data_split = ALL_RES_MAESTRO.groupby(['model', 'split'])[all_metrics].mean()
MAESTRO_RES_data_split = sort_index(MAESTRO_RES_data_split, index_level='split')
MAESTRO_RES_data_split = sort_index(
    MAESTRO_RES_data_split, index_level='model')
MAESTRO_RES_data_split

# sort by model only
MAESTRO_RES_data = ALL_RES_MAESTRO.groupby(['model'])[all_metrics].mean()
MAESTRO_RES_data
# go from wide to long
MAESTRO_RES_table = MAESTRO_RES_data.transpose()
# sort and rename
MAESTRO_RES_table = MAESTRO_RES_table[['oaf', 'kong', 'T5']]
metrics_names_tables = {
    'f_f': 'Frame F1',
    'f_no': 'Note Offset F1',
    'f_nov': 'Note Offset Velocity F1',
    'melody_ioi_corr': 'Melody IOI',
    'acc_ioi_corr': 'Accompaniment IOI',
    'melody_kor_corr_64': 'Melody KOR',
    'bass_kor_corr_64': 'Bass KOR',
    'ratio_kor_corr_64': 'Ratio KOR',
    'cd_corr': 'Cloud Diameter',
    'cm_corr': 'Cloud Momentum',
    'ts_corr': 'Tensile Strain',
    'dyn_corr': 'Dynamics',
    'peamt': 'PEAMT'
}
MAESTRO_RES_table.index = MAESTRO_RES_table.index.map(metrics_names_tables)
# save to latex
MAESTRO_RES_table.to_latex(os.path.join(TABLES_DIR, 'tab2_metrics_overview_MAESTRO_subset.tex'), escape=False, float_format="%.4f")

MAESTRO_RES_table

model,oaf,kong,T5
Frame F1,0.870965,0.913848,0.704802
Note Offset F1,0.616745,0.873618,0.635773
Note Offset Velocity F1,0.591654,0.858691,0.630874
Melody IOI,0.237703,0.548081,0.221730
Accompaniment IOI,0.216814,0.367910,0.432896
Melody KOR,0.405668,0.741523,0.282514
Bass KOR,0.263814,0.696660,0.267218
Ratio KOR,0.424690,0.693797,0.309429
Cloud Diameter,0.724036,0.830093,0.747168
Cloud Momentum,0.246073,0.225039,0.167105


In [14]:
# Table 3: Benchmark Results per model, split, and recording

# get mean frame fscore per model, split, and recording
frame_mean = frame_res.groupby(['model', 'split', 'recording'])[['f_f']].mean()
frame_mean = sort_index(frame_mean, 'model')
frame_mean = sort_index(frame_mean, 'split')
frame_mean = sort_index(frame_mean, 'recording')
frame_mean_values = frame_mean.values

# get mean note_offset fscore per model, split, and recording
note_offset_mean = note_offset_res.groupby(
    ['model', 'split', 'recording'])[['f_no']].mean()
note_offset_mean = sort_index(note_offset_mean, 'model')
note_offset_mean = sort_index(note_offset_mean, 'split')
note_offset_mean = sort_index(note_offset_mean, 'recording')
note_offset_mean_values = note_offset_mean.values

# get mean note_offset_velocity fscore per model, split, and recording
note_offset_velocity_mean = note_offset_velocity_res.groupby(
    ['model', 'split', 'recording'])[['f_nov']].mean()
note_offset_velocity_mean = sort_index(note_offset_velocity_mean, 'model')
note_offset_velocity_mean = sort_index(note_offset_velocity_mean, 'split')
note_offset_velocity_mean = sort_index(note_offset_velocity_mean, 'recording')
note_offset_velocity_mean_values = note_offset_velocity_mean.values

# combine them in one df
model_split_rec_res = frame_mean.index.to_frame(index=False)
model_split_rec_res['frame'] = frame_mean_values
model_split_rec_res['note_offset'] = note_offset_mean_values
model_split_rec_res['note_offset_velocity'] = note_offset_velocity_mean_values
model_split_rec_res

# pivot the table for the paper
model_split_rec_res_wide = model_split_rec_res.pivot(index=['split', 'recording'], columns='model', values=[
    'frame', 'note_offset', 'note_offset_velocity'])
# sort columns and index
columns_sorted = [
    ('frame',  'oaf'),
    ('frame', 'kong'),
    ('frame',   'T5'),
    ('note_offset',  'oaf'),
    ('note_offset', 'kong'),
    ('note_offset',   'T5'),
    ('note_offset_velocity',  'oaf'),
    ('note_offset_velocity', 'kong'),
    ('note_offset_velocity',   'T5')]
model_split_rec_res_wide = model_split_rec_res_wide[columns_sorted]

model_split_rec_res_wide = sort_index(model_split_rec_res_wide, 'split')
model_split_rec_res_wide = sort_index(model_split_rec_res_wide, 'recording')

# drop batik results from table
model_split_rec_res_wide = model_split_rec_res_wide.drop([('Batik',      'batik'),
                                                          ('Batik', 'disklavier')])
# save to latex
model_split_rec_res_wide.to_latex(os.path.join(
    TABLES_DIR, 'tab3_benchmark_per_split_per_recording.tex'), float_format="%.4f")

model_split_rec_res_wide

frame                     note_offset            \
model                       oaf      kong        T5         oaf      kong   
split      recording                                                        
train      maestro     0.880704  0.920717  0.726247    0.618337  0.889925   
           disklavier  0.818479  0.850774  0.615717    0.526871  0.738400   
validation maestro     0.840409  0.893618  0.654645    0.649218  0.861709   
           disklavier  0.769609  0.867827  0.609309    0.553900  0.814155   
test       maestro     0.852710  0.900245  0.655185    0.593150  0.821465   
           disklavier  0.804895  0.853020  0.604790    0.498850  0.697925   

                                note_offset_velocity                      
model                        T5                  oaf      kong        T5  
split      recording                                                      
train      maestro     0.634979             0.592929  0.875636  0.630817  
           disklavier  0.513154             0.485343  0.665978  0.466278  
validation maestro     0.711745             0.623573  0.847109  0.706273  
           disklavier  0.657045             0.516136  0.747955  0.603127  
test       maestro     0.596845             0.569510  0.804060  0.589610  
           disklavier  0.513485             0.460710  0.630395  0.462425